In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1650956947191_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = sqlContext.read.csv('s3a://linear-regression-mlc/train.csv', header=True, inferSchema=True)
# Using a smaller dataset of 10M rows
df = df.limit(1_000_000)
df = df.dropna()
df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key: timestamp, fare_amount: double, pickup_datetime: string, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, passenger_count: int]

In [3]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+--------------------+------------------+-----------------+-----------------+-----------------+---------------+
|                key|fare_amount|     pickup_datetime|  pickup_longitude|  pickup_latitude|dropoff_longitude| dropoff_latitude|passenger_count|
+-------------------+-----------+--------------------+------------------+-----------------+-----------------+-----------------+---------------+
|2010-02-08 22:14:00|        6.5|2010-02-08 22:14:...|        -73.948425|        40.774048|       -73.965362|        40.757485|              1|
|2010-06-18 15:26:00|       13.3|2010-06-18 15:26:...|        -73.976297|        40.789097|       -74.004615|        40.742228|              5|
|2014-11-18 20:33:40|        6.5|2014-11-18 20:33:...|        -73.982605|        40.762357|       -73.995942|        40.753806|              1|
|2011-06-14 20:38:00|       20.5|2011-06-14 20:38:...|        -74.009922|        40.720078|       -73.955662|        40.783623|         

In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
inputCols = [
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count'
]
assembler = VectorAssembler(inputCols = inputCols , outputCol = 'features')
dataset = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'fare_amount')
model = lr.fit(dataset)
summary = model.evaluate(dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.000245481100044298

In [9]:
df.agg({'pickup_latitude':'max'}).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(max(pickup_latitude)=2009.26628)]

In [10]:
#NYC lies between 73 and 75 degrees West, and 40 and 42 degrees north

TOP, BOTTOM, LEFT, RIGHT = 42, 40 ,-75, -73
df = df.filter(df['pickup_latitude'] >= BOTTOM)
df = df.filter(df['pickup_latitude'] <= TOP)
df = df.filter(df['pickup_longitude'] <= RIGHT)
df = df.filter(df['pickup_longitude'] >= LEFT)

df = df.filter(df['dropoff_latitude'] >= BOTTOM)
df = df.filter(df['dropoff_latitude'] <= TOP)
df = df.filter(df['dropoff_longitude'] <= RIGHT)
df = df.filter(df['dropoff_longitude'] >= LEFT)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

978872

In [12]:
df.agg({'passenger_count':'min'}).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(min(passenger_count)=0)]

In [14]:
df =df.filter(df['passenger_count']>0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df.agg({'fare_amount':'min'}).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(min(fare_amount)=-52.0)]

In [16]:
df =df.filter(df['fare_amount']>0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

975428

In [18]:
inputCols = [
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count'
]
assembler = VectorAssembler(inputCols = inputCols, outputCol = 'featuresClean')
dataset = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
lr = LinearRegression(featuresCol = 'featuresClean', labelCol = 'fare_amount')
model = lr.fit(dataset)
summary = model.evaluate(dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.250888962495586

In [21]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+--------------------+------------------+-----------------+-----------------+-----------------+---------------+
|                key|fare_amount|     pickup_datetime|  pickup_longitude|  pickup_latitude|dropoff_longitude| dropoff_latitude|passenger_count|
+-------------------+-----------+--------------------+------------------+-----------------+-----------------+-----------------+---------------+
|2010-02-08 22:14:00|        6.5|2010-02-08 22:14:...|        -73.948425|        40.774048|       -73.965362|        40.757485|              1|
|2010-06-18 15:26:00|       13.3|2010-06-18 15:26:...|        -73.976297|        40.789097|       -74.004615|        40.742228|              5|
|2014-11-18 20:33:40|        6.5|2014-11-18 20:33:...|        -73.982605|        40.762357|       -73.995942|        40.753806|              1|
|2011-06-14 20:38:00|       20.5|2011-06-14 20:38:...|        -74.009922|        40.720078|       -73.955662|        40.783623|         

In [22]:
df.select(('pickup_datetime')).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|pickup_datetime        |
+-----------------------+
|2010-02-08 22:14:00 UTC|
|2010-06-18 15:26:00 UTC|
|2014-11-18 20:33:40 UTC|
|2011-06-14 20:38:00 UTC|
|2011-06-21 07:08:16 UTC|
|2009-02-10 19:17:09 UTC|
|2009-08-07 22:03:00 UTC|
|2009-10-14 02:38:00 UTC|
|2012-07-13 15:01:23 UTC|
|2013-11-22 00:09:00 UTC|
|2014-10-30 11:08:00 UTC|
|2012-02-16 04:11:14 UTC|
|2012-12-04 15:46:10 UTC|
|2013-05-11 23:57:00 UTC|
|2010-12-08 06:27:46 UTC|
|2011-03-29 02:46:00 UTC|
|2014-01-13 22:43:00 UTC|
|2014-04-30 18:09:00 UTC|
|2015-03-10 18:33:13 UTC|
|2012-01-10 15:53:00 UTC|
+-----------------------+
only showing top 20 rows

In [23]:
df=df.withColumn('datetime',df['pickup_datetime'].substr(0,19))
df.select('datetime').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|datetime           |
+-------------------+
|2010-02-08 22:14:00|
|2010-06-18 15:26:00|
|2014-11-18 20:33:40|
|2011-06-14 20:38:00|
|2011-06-21 07:08:16|
|2009-02-10 19:17:09|
|2009-08-07 22:03:00|
|2009-10-14 02:38:00|
|2012-07-13 15:01:23|
|2013-11-22 00:09:00|
|2014-10-30 11:08:00|
|2012-02-16 04:11:14|
|2012-12-04 15:46:10|
|2013-05-11 23:57:00|
|2010-12-08 06:27:46|
|2011-03-29 02:46:00|
|2014-01-13 22:43:00|
|2014-04-30 18:09:00|
|2015-03-10 18:33:13|
|2012-01-10 15:53:00|
+-------------------+
only showing top 20 rows

In [24]:
from pyspark.sql.functions import to_timestamp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df = df.withColumn('timestamp', to_timestamp(df['datetime']))
df.select('timestamp').show(truncate =False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|timestamp          |
+-------------------+
|2010-02-08 22:14:00|
|2010-06-18 15:26:00|
|2014-11-18 20:33:40|
|2011-06-14 20:38:00|
|2011-06-21 07:08:16|
|2009-02-10 19:17:09|
|2009-08-07 22:03:00|
|2009-10-14 02:38:00|
|2012-07-13 15:01:23|
|2013-11-22 00:09:00|
|2014-10-30 11:08:00|
|2012-02-16 04:11:14|
|2012-12-04 15:46:10|
|2013-05-11 23:57:00|
|2010-12-08 06:27:46|
|2011-03-29 02:46:00|
|2014-01-13 22:43:00|
|2014-04-30 18:09:00|
|2015-03-10 18:33:13|
|2012-01-10 15:53:00|
+-------------------+
only showing top 20 rows

In [26]:
from pyspark.sql.functions import year,month,dayofweek, hour

df = df.withColumn('year', year(df['timestamp']))
df = df.withColumn('month', month(df['timestamp']))
df = df.withColumn('dayofweek', dayofweek(df['timestamp']))
df = df.withColumn('hour', hour(df['timestamp']))
df.select('year','month', 'dayofweek', 'hour').show(truncate =  False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---------+----+
|year|month|dayofweek|hour|
+----+-----+---------+----+
|2010|2    |2        |22  |
|2010|6    |6        |15  |
|2014|11   |3        |20  |
|2011|6    |3        |20  |
|2011|6    |3        |7   |
|2009|2    |3        |19  |
|2009|8    |6        |22  |
|2009|10   |4        |2   |
|2012|7    |6        |15  |
|2013|11   |6        |0   |
|2014|10   |5        |11  |
|2012|2    |5        |4   |
|2012|12   |3        |15  |
|2013|5    |7        |23  |
|2010|12   |4        |6   |
|2011|3    |3        |2   |
|2014|1    |2        |22  |
|2014|4    |4        |18  |
|2015|3    |3        |18  |
|2012|1    |3        |15  |
+----+-----+---------+----+
only showing top 20 rows

In [27]:
inputCols = [
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count', 'year', 'month', 'dayofweek' , 'hour'
]
assembler = VectorAssembler(inputCols = inputCols, outputCol = 'featuresCleanWithDate')
dataset = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
lr = LinearRegression(featuresCol = 'featuresCleanWithDate', labelCol = 'fare_amount')
model = lr.fit(dataset)
summary = model.evaluate(dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
from pyspark.sql.functions import from_utc_timestamp
df = df.withColumn('NYTime', from_utc_timestamp(df['timestamp'], 'EST'))
df.select('NYTime').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|             NYTime|
+-------------------+
|2010-02-08 17:14:00|
|2010-06-18 10:26:00|
|2014-11-18 15:33:40|
|2011-06-14 15:38:00|
|2011-06-21 02:08:16|
|2009-02-10 14:17:09|
|2009-08-07 17:03:00|
|2009-10-13 21:38:00|
|2012-07-13 10:01:23|
|2013-11-21 19:09:00|
|2014-10-30 06:08:00|
|2012-02-15 23:11:14|
|2012-12-04 10:46:10|
|2013-05-11 18:57:00|
|2010-12-08 01:27:46|
|2011-03-28 21:46:00|
|2014-01-13 17:43:00|
|2014-04-30 13:09:00|
|2015-03-10 13:33:13|
|2012-01-10 10:53:00|
+-------------------+
only showing top 20 rows

In [30]:
from pyspark.sql.functions import year,month,dayofweek, hour

df = df.withColumn('year', year(df['NYTime']))
df = df.withColumn('month', month(df['NYTime']))
df = df.withColumn('dayofweek', dayofweek(df['NYTime']))
df = df.withColumn('hour', hour(df['NYTime']))
df.select('year','month', 'dayofweek', 'hour').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---------+----+
|year|month|dayofweek|hour|
+----+-----+---------+----+
|2010|    2|        2|  17|
|2010|    6|        6|  10|
|2014|   11|        3|  15|
|2011|    6|        3|  15|
|2011|    6|        3|   2|
|2009|    2|        3|  14|
|2009|    8|        6|  17|
|2009|   10|        3|  21|
|2012|    7|        6|  10|
|2013|   11|        5|  19|
|2014|   10|        5|   6|
|2012|    2|        4|  23|
|2012|   12|        3|  10|
|2013|    5|        7|  18|
|2010|   12|        4|   1|
|2011|    3|        2|  21|
|2014|    1|        2|  17|
|2014|    4|        4|  13|
|2015|    3|        3|  13|
|2012|    1|        3|  10|
+----+-----+---------+----+
only showing top 20 rows

In [31]:
inputCols = [
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count', 'year', 'month', 'dayofweek' , 'hour'
]
assembler = VectorAssembler(inputCols = inputCols, outputCol = 'featuresCleanWithDate')
dataset = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
lr = LinearRegression(featuresCol = 'featuresCleanWithDate', labelCol = 'fare_amount')
model = lr.fit(dataset)
summary = model.evaluate(dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.26449290061767994

In [34]:
x1 = df['pickup_longitude']
y1 = df['pickup_latitude']
x2 = df['dropoff_longitude']
y2 = df['dropoff_latitude']

from pyspark.sql.functions import abs as psabs
df = df.withColumn('l1', psabs(x1 - x2) + psabs(y1 - y2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
inputCols = [
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count', 'year', 'month', 'dayofweek' , 'hour', 'l1'
]
assembler = VectorAssembler(inputCols = inputCols, outputCol = 'featuresCleanWithDatel1')
dataset = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
lr = LinearRegression(featuresCol = 'featuresCleanWithDatel1', labelCol = 'fare_amount')
model = lr.fit(dataset)
summary = model.evaluate(dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6794746273189293

In [38]:
train, test = df.randomSplit([0.66, 0.33])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
trainDataset = assembler.transform(train)
testDataset = assembler.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
lr = LinearRegression(featuresCol = 'featuresCleanWithDatel1', labelCol = 'fare_amount')
model = lr.fit(trainDataset)
summary = model.evaluate(testDataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6811101355366389